## Case for Eco.
> 2023-01-29
### 1.import packages

In [1]:
import sys,os,re
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
sys.path.append("../tools/")
import tools.funclib as funclib
import numpy as np
import pandas as pd
from keras.models import load_model
from tools.Attention import Attention
from tkinter import _flatten
import config as cfg
import tools.embedding_esm as ebdesm
from pandarallel import pandarallel 
pandarallel.initialize()

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 52 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2. Read pnas eco.in fasta file

In [2]:
# 读取一次预测结果
data_eco =pd.read_excel(cfg.RESULTSDIR+'ecoli/ecolicase_results.xlsx')

# 读取pnas 数据
pnas_s4 = pd.read_excel(cfg.RESULTSDIR+'ecoli/pnas.2211197119.sd04.xlsx')

# inlist=list(set(pnas_s4[~pnas_s4.gene_id.isnull()].gene_id))
# inlist = ['b4054']
# data_eco = data_eco[data_eco.id.isin(inlist)].reset_index(drop=True)
sprot_leatest = pd.read_feather(cfg.FILE_LATEST_SPROT_FEATHER)

data_eco.head(3)

,id,gene_name,protein_name,ec_number,seq,ec_deepec,id_uniprot,ec_ecrecer,date_integraged,date_sequence_update,date_annotation_update,res_type,correct_deepec,correct_ecrecer,function_counts
0,b0001,thrL,thr operon leader peptide,-,MKRISTTITTTITITTGNGAG,-,P0AD87,-,1986-07-21,1986-07-21,2021-09-29,db_match,True,True,1
1,b0002,thrA,Bifunctional aspartokinase/homoserine dehydrog...,"1.1.1.3, 2.7.2.4",MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,"1.1.1.3, 2.7.2.4",P00561,"1.1.1.3, 2.7.2.4",1986-07-21,2003-08-29,2021-09-29,db_match,True,True,2
2,b0003,thrB,homoserine kinase,2.7.1.39,MVKVYAPASSANMSVGFDVLGAAVTPVDGALLGDVVTVEAAETFSL...,2.7.1.39,P00547,2.7.1.39,1986-07-21,1993-04-01,2020-12-02,db_match,True,True,1


## 3. Embedding

In [3]:
# final_rep0, final_rep32, final_rep33 = ebdesm.get_rep_multi_sequence(data_eco[['id','seq']])
# final_rep32.head(2)
final_rep32 = pd.read_feather(cfg.RESULTSDIR+'ecoli/rep32.feather')

## 4.读取测试数据

In [4]:
# embeddin 方法
EMBD_METHO='esm32'
EMBD_WH = 1280
TOPK=5

ecs_2 = [True, False]
ecs_8 = [1,2,3,4,5,6,7,8]
label_model_ec = pd.read_feather(f'{cfg.MODELDIR}/task3_labels.feather').label_multi.to_list()

test_feature = final_rep32.iloc[:, 1:]
test_feature =np.reshape(np.array(test_feature),(test_feature.shape[0],1,EMBD_WH))
test_res = pd.DataFrame(final_rep32.id.copy().astype('str'))

## 5. DMLF [task1, task2, task3]

In [5]:
# task1
model_t1 = load_model(f'{cfg.MODELDIR}/task1_2022.h5', custom_objects={"Attention": Attention},compile=False)
predicted = model_t1.predict(test_feature)
test_res['dmlf_isenzyme'] = np.argmax(predicted, axis=-1).astype('bool')

# Task2
list_func = list()
model_t2 = load_model(f'{cfg.MODELDIR}/task2_model_2022_label8.h5',custom_objects={"Attention": Attention},compile=False)
predicted = model_t2.predict(test_feature)

for i in range(predicted.shape[0]):
    list_func.append(sorted(dict(zip(set(ecs_8),predicted[i])).items(),key = lambda x:x[1], reverse = True)[0][0])
test_res['dmlf_functions']=list_func


#Task3
model_t3 = load_model(f'{cfg.MODELDIR}/task3_esm32_2022.h5',custom_objects={"Attention": Attention},compile=False)
predicted = model_t3.predict(test_feature)
test_res['sorted_ec'] = pd.DataFrame(predicted).parallel_apply(lambda x: sorted(dict(zip((label_model_ec), x.to_list())).items(),key = lambda x:x[1], reverse = True), axis=1)
test_res['ec_dmlf']=test_res.parallel_apply(lambda x: (list(dict(x.sorted_ec[0:x.dmlf_functions]).keys())), axis=1)
test_res['ec_dmlf']=test_res.parallel_apply(lambda x: '-' if x.dmlf_isenzyme ==False else ','.join(x.ec_dmlf), axis=1)
test_res['ec_dmlf_topk']=test_res.parallel_apply(lambda x: x.sorted_ec[0:TOPK], axis=1)

save_res =data_eco.merge(test_res, on='id', how='left')

136/136 [==============================] - 1s 1ms/step


In [6]:
save_res

,id,gene_name,protein_name,ec_number,seq,ec_deepec,id_uniprot,ec_ecrecer,date_integraged,date_sequence_update,date_annotation_update,res_type,correct_deepec,correct_ecrecer,function_counts,dmlf_isenzyme,dmlf_functions,sorted_ec,ec_dmlf,ec_dmlf_topk
0,b0001,thrL,thr operon leader peptide,-,MKRISTTITTTITITTGNGAG,-,P0AD87,-,1986-07-21,1986-07-21,2021-09-29,db_match,True,True,1,False,1,"[(3.4.21.-, 0.09198837727308273), (3.1.1.74, 0...",-,"[(3.4.21.-, 0.09198837727308273), (3.1.1.74, 0..."
1,b0002,thrA,Bifunctional aspartokinase/homoserine dehydrog...,"1.1.1.3, 2.7.2.4",MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,"1.1.1.3, 2.7.2.4",P00561,"1.1.1.3, 2.7.2.4",1986-07-21,2003-08-29,2021-09-29,db_match,True,True,2,True,2,"[(2.7.2.4, 0.9984791874885559), (1.1.1.3, 0.99...","2.7.2.4,1.1.1.3","[(2.7.2.4, 0.9984791874885559), (1.1.1.3, 0.99..."
2,b0003,thrB,homoserine kinase,2.7.1.39,MVKVYAPASSANMSVGFDVLGAAVTPVDGALLGDVVTVEAAETFSL...,2.7.1.39,P00547,2.7.1.39,1986-07-21,1993-04-01,2020-12-02,db_match,True,True,1,True,1,"[(2.7.1.39, 0.9998766183853149), (2.7.1.148, 0...",2.7.1.39,"[(2.7.1.39, 0.9998766183853149), (2.7.1.148, 0..."
3,b0004,thrC,L-threonine synthase,4.2.3.1,MKLYNLKDHNEQVSFAQAVTQGLGKNQGLFFPHDLPEFSLTEIDEM...,4.2.3.1,P00934,4.2.3.1,1986-07-21,1986-07-21,2021-06-02,db_match,True,True,1,True,1,"[(4.2.3.1, 0.9761539101600647), (4.2.3.-, 0.00...",4.2.3.1,"[(4.2.3.1, 0.9761539101600647), (4.2.3.-, 0.00..."
4,b0005,yaaX,DUF2502 family putative periplasmic protein,-,MKKMQSIVLALSLVLVAPMAAQAAEITLVPSVKLQIGDRDNRGYYW...,-,P75616,-,2000-05-30,1997-02-01,2021-06-02,db_match,True,True,1,False,1,"[(3.4.-.-, 0.18714982271194458), (3.1.27.-, 0....",-,"[(3.4.-.-, 0.18714982271194458), (3.1.27.-, 0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4317,b4706,iroK,3-hydroxypropionic acid resistance peptide,-,MKPALRDFIAIVQERLASVTA,-,U3PVA8,-,2014-06-11,2013-12-11,2020-10-07,db_match,True,True,1,False,1,"[(1.3.7.7, 0.14355075359344482), (5.1.1.1, 0.1...",-,"[(1.3.7.7, 0.14355075359344482), (5.1.1.1, 0.1..."
4318,b4708,insI2,IS30 transposase,-,ENTNGLIRQYFPKKTCLAQYTQHELDLVAAQLNNRPRKTLKFKTPK...,-,P0CF88,-,2010-05-18,2010-05-18,2021-09-29,blast_match,True,True,1,False,1,"[(3.1.11.6, 0.000813148042652756), (3.2.2.-, 0...",-,"[(3.1.11.6, 0.000813148042652756), (3.2.2.-, 0..."
4319,b4709,insA9,IS1 transposase A,-,MASVSISCPSCSATDGVVRNGKSTAGHQRYLCSHCRKTWQLQFTYT...,-,P0ADH1,-,1986-07-21,2005-12-06,2021-04-07,db_match,True,True,1,False,1,"[(3.1.-.-, 0.3655907213687897), (3.6.4.-, 0.02...",-,"[(3.1.-.-, 0.3655907213687897), (3.6.4.-, 0.02..."
4320,b4710,insB9,IS1 transposase B,-,MPGNSPHYGRWPQHDFTSLKKLRPQSVTSRIQPGSDVIVCAEMDEQ...,-,P59843,-,2003-09-09,2003-09-09,2020-12-02,db_match,True,True,1,False,1,"[(2.7.7.49, 0.009339512325823307), (3.4.23.16,...",-,"[(2.7.7.49, 0.009339512325823307), (3.4.23.16,..."


## 6. 处理推荐

In [6]:
# 加载EC数据
eclist = pd.read_csv(cfg.DATADIR+'explorenz/tb_ec_expasy_202211161459.csv')
TOPK=5
# 匹配EC号到反应
def handle_topk(item):
    counter = 0
    res_str =''
    item = list(set([subitem[0] for subitem in item]))

    if len(item)<1:
        return '-'
    else:
        for i in item:
            counter = counter +1
            # if '-' in i:
            #     counter = counter -1
            #     continue
            
            if counter >TOPK:
                return res_str
            
            res_str = res_str + f'[{counter} {eclist[eclist.ec==i].reaction.values[0] if len (eclist[eclist.ec==i])>0 else ""}],'
            return res_str
        

save_res['top_message'] = save_res.ec_dmlf_topk.parallel_apply(lambda x: handle_topk(item=x))
save_res = save_res[['id','gene_name','protein_name', 'ec_number','ec_deepec','id_uniprot', 'ec_ecrecer', 'ec_dmlf', 'ec_dmlf_topk', 'top_message']]
save_res.head(3)
# save_res.head(5).ec_dmlf_topk.apply(lambda x: handle_topk(item=x))

,id,gene_name,protein_name,ec_number,ec_deepec,id_uniprot,ec_ecrecer,ec_dmlf,ec_dmlf_topk,top_message
0,b0001,thrL,thr operon leader peptide,-,-,P0AD87,-,-,"[(3.4.21.-, 0.09198837727308273), (3.1.1.74, 0...","[1 ],"
1,b0002,thrA,Bifunctional aspartokinase/homoserine dehydrog...,"1.1.1.3, 2.7.2.4","1.1.1.3, 2.7.2.4",P00561,"1.1.1.3, 2.7.2.4","2.7.2.4,1.1.1.3","[(2.7.2.4, 0.9984791874885559), (1.1.1.3, 0.99...",[1 3-phospho-D-glycerate + NAD(+) = 3-phosphon...
2,b0003,thrB,homoserine kinase,2.7.1.39,2.7.1.39,P00547,2.7.1.39,2.7.1.39,"[(2.7.1.39, 0.9998766183853149), (2.7.1.148, 0...",[1 D-glycero-alpha-D-manno-heptose 7-phosphate...


In [7]:
save_res.to_excel(cfg.RESULTSDIR+'ecoli/case_ecolin_dmlf_20130215.xlsx')

## 7. 处理Pnas中gap数据

In [7]:


# 匹配反应与基因

map_reaction_gene=pnas_s4[['Alternative', 'CANDIDATE_GENES_rxn1', 'CANDIDATE_GENES_rxn2', 'CANDIDATE_GENES_rxn3', 'CANDIDATE_GENES_rxn4']]
map_reaction_gene = map_reaction_gene[~map_reaction_gene.Alternative.isnull()]
map_reaction_gene

atlas_map_results=[]
for index, itemrow in map_reaction_gene.iterrows():
    atlas_reactions = itemrow['Alternative'].split('|')
    subindex=1
    for item in atlas_reactions:
        atlas_map_results = atlas_map_results +[[item, itemrow[subindex]]]
        subindex = subindex + 1
    # print(atlas_reactions)


atlas_map_results=pd.DataFrame(atlas_map_results, columns=['atlas_ractions', 'gene_name'])
atlas_map_results

# 按反应分组
atlas_map_results_2 = []
for name, group in atlas_map_results.groupby(by='atlas_ractions'):
    if not pd.isnull(group.gene_name.values).all():
        atlas_map_results_2 = atlas_map_results_2 +[[name, ','.join(list(set(group.gene_name)))]]
    else:
        atlas_map_results_2 = atlas_map_results_2 +[[name, '']]
    # print(name, ','.join(group.gene_name.to_list()))

atlas_map_results=pd.DataFrame(atlas_map_results_2, columns=['atlas_ractions', 'gene_name'])
atlas_map_results['id']=atlas_map_results.atlas_ractions.apply(lambda x: int(x.replace('rat','')))

# 读取Altas反应数据
atlas = pd.read_csv(cfg.RESULTSDIR+'ecoli/atlas_0.2.txt',sep=';')
atlas = atlas[['id','REACTION','ATLAS', 'REACTION RULE']].rename(columns={'ATLAS':'KEGG_ID'})

# 匹配Atlas 反应方程式
atlas_map_results = atlas_map_results.merge(atlas, on='id', how='left')
atlas_map_results = atlas_map_results[['atlas_ractions','id','gene_name','REACTION','KEGG_ID','REACTION RULE']].rename(columns={'id':'rat_id'})
atlas_map_results

#KEGG 化合物
keggcpd=pd.read_csv(cfg.RESULTSDIR+'ecoli/kegg_cpds.tsv', sep='\t')

# 转化方程式为可读取格式
def format_keeg_cpd(eq):
    if str(eq)!='nan':
        cpd_list = eq.split('<=>')
        cpd_list = _flatten([item.split('+') for item in cpd_list])
        cpd_list = _flatten([item.split(' ') for item in cpd_list])
        cpd_list = list([re.sub(r'\(\d\)',r'',x) for x in cpd_list  if 'C' in x])

        for item in cpd_list:
            eq = eq.replace(item, ' '+keggcpd[keggcpd.cpd_id==f'cpd:{item}'].cpd_name.values[0].split(';')[0]+' ')
        return eq
    else:
        return ''
    
atlas_map_results['atlas_reactionformula']=atlas_map_results.REACTION.apply(lambda x: format_keeg_cpd(x))
atlas_map_results

,atlas_ractions,rat_id,gene_name,REACTION,KEGG_ID,REACTION RULE,atlas_reactionformula
0,rat102090,102090,galK,C01187+C04188<=>C03089+C04478,NaN,2.7.8.-(rev),3-Deoxy-D-manno-octulosonate + S-Methyl-5-thi...
1,rat121283,121283,atoA,C00582+C02730<=>C03160+C07086,NaN,2.8.3.-(rev),Phenylacetyl-CoA + 2-Succinylbenzoate <=> 2-S...
2,rat12707,12707,hcaE,C00001+C00006+C00212<=>C00005+C00007+C00080+C0...,NaN,1.14.13.-|1.14.13.-(rev),H2O + NADP+ + Adenosine <=> NADPH + Oxygen + ...
3,rat12713,12713,hcaE,C00001+C00006+C00483<=>C00005+C00007+C00080+C0...,NaN,1.14.13.-|1.14.13.-(rev),H2O + NADP+ + Tyramine <=> NADPH + Oxygen + H...
4,rat129326,129326,riha,C00001+C01103<=>C00117+C00295,NaN,3.2.2.-|3.2.2.-(rev),H2O + Orotidine 5'-phosphate <=> D-Ribose 5-p...
...,...,...,...,...,...,...,...
73,rat82788,82788,galK,C00263+C11472<=>C01102+C07836,NaN,2.7.8.-(rev),L-Homoserine + D-glycero-beta-D-manno-Heptose...
74,rat8817,8817,mqo,C00001+C00007+C03406<=>C00027+C00036+C00062,NaN,1.4.3.-,H2O + Oxygen + N-(L-Arginino)succinate <=> Hy...
75,rat9163,9163,glcD,C00007+C00037<=>C00027+C15809,R07463,1.4.3.-|1.4.3.-(rev),Oxygen + Glycine <=> Hydrogen peroxide + Imin...
76,rat9190,9190,,C00003+C00148<=>C00004+C00080+C03912,R01248,1.5.1.-|1.5.1.-(rev),NAD+ + L-Proline <=> NADH + H+ + (S)-1-Pyrrol...


In [8]:
#展开gene

gene_35 = atlas_map_results[atlas_map_results.gene_name!=''].copy()

res_set = []
for index, iterrow in gene_35.iterrows():
    gene_list=  iterrow.gene_name.split(',')
    if len(gene_list) <2:
        res_set = res_set +[iterrow.to_list()]
    else:
        for gene_item in gene_list:
            temp_array = iterrow.to_list()
            temp_array[2]= gene_item
            res_set = res_set +[temp_array]


gene_35 = pd.DataFrame(res_set, columns=gene_35.columns.values)

gene_35['gene_name'] = gene_35['gene_name'].str.lower()
save_res['gene_name'] =  save_res['gene_name'].str.lower()
gene_35 = gene_35.merge(save_res, left_on='gene_name', right_on='gene_name', how='left')


In [9]:
writer = pd.ExcelWriter(cfg.RESULTSDIR+ 'ecoli/pnas_78_reaction_35_gene_case_3.xlsx')
workbook  = writer.book
atlas_map_results.to_excel(writer, sheet_name="78_reactions", index=False)
gene_35.to_excel(writer, sheet_name="35_genes", index=False)

writer.close()